In [31]:
import pandas as pd
import quandl
import re
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
import itertools
import unicodedata


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 90)
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', 26)
pd.set_option('display.width', 200)

api_key = 'HxxAsWWP1AJofAW8u261'
quandl.ApiConfig.api_key = api_key

In [2]:
"""
US_States = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States")
US_States = US_States[1].iloc[:,0:2]
US_States.columns = ["State", "Abbreviation"]

US_States.State = US_States.State.apply(lambda x: x.strip('[D]'))
# a link to a note in wikipedia that is irrelevant to our use-case, so it is removed.

US_States.set_index('State', inplace=True)
"""
# US_States

'\nUS_States = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States")\nUS_States = US_States[1].iloc[:,0:2]\nUS_States.columns = ["State", "Abbreviation"]\n\nUS_States.State = US_States.State.apply(lambda x: x.strip(\'[D]\'))\n# a link to a note in wikipedia that is irrelevant to our use-case, so it is removed.\n\nUS_States.set_index(\'State\', inplace=True)\n'

In [3]:
"""
for index, row in US_States.iterrows():
    if index == 'Alabama':
        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)
        Temp.columns = [index, index]
        SA_Data = pd.DataFrame(Temp.iloc[:,0], index = Temp.index)
        NSA_Data = pd.DataFrame(Temp.iloc[:,1], index = Temp.index)
    else:
        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)
        Temp.columns = [index, index]
        SA_Data[index] = Temp.iloc[:,0]
        NSA_Data[index] = Temp.iloc[:,1]
"""
# Later we pickle the data so this gathering step is obsolete
# SA_Data

'\nfor index, row in US_States.iterrows():\n    if index == \'Alabama\':\n        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)\n        Temp.columns = [index, index]\n        SA_Data = pd.DataFrame(Temp.iloc[:,0], index = Temp.index)\n        NSA_Data = pd.DataFrame(Temp.iloc[:,1], index = Temp.index)\n    else:\n        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)\n        Temp.columns = [index, index]\n        SA_Data[index] = Temp.iloc[:,0]\n        NSA_Data[index] = Temp.iloc[:,1]\n'

In [4]:
# NSA_Data

In [5]:
"""
NSA_Data.to_pickle('Pickles/NSA_Data.pickle')
SA_Data.to_pickle('Pickles/SA_Data.pickle')
US_States.to_pickle('Pickles/US_States.pickle')
"""

"\nNSA_Data.to_pickle('Pickles/NSA_Data.pickle')\nSA_Data.to_pickle('Pickles/SA_Data.pickle')\nUS_States.to_pickle('Pickles/US_States.pickle')\n"

In [6]:
NSA_Data = pd.read_pickle('Pickles/NSA_Data.pickle')
SA_Data = pd.read_pickle('Pickles/SA_Data.pickle')
US_States = pd.read_pickle('Pickles/US_States.pickle')

In [7]:
"""
Hurricanes = pd.read_html("https://en.wikipedia.org/wiki/List_of_Category_5_Atlantic_hurricanes",header=[0],flavor='bs4')
Hurricanes = Hurricanes[1]
# Hurricanes.set_index("Name", inplace = True)
Hurricanes.to_pickle("Pickles/Hurricanes.pickle")
"""

'\nHurricanes = pd.read_html("https://en.wikipedia.org/wiki/List_of_Category_5_Atlantic_hurricanes",header=[0],flavor=\'bs4\')\nHurricanes = Hurricanes[1]\n# Hurricanes.set_index("Name", inplace = True)\nHurricanes.to_pickle("Pickles/Hurricanes.pickle")\n'

## Changes to be made:
1. Getting both dates into the "yyyy-mm" format, as the HPI data includes only the last day of the month.
1. Getting the Damage information from wikipedia
1. Narrowing down the Areas affected to those in the US and specifically which states involved which hurricane (Manual labour)
1. Assigining an "Overall Strength" feature which may combine different damage reports.

In [8]:
Hurricanes = pd.read_pickle("Pickles/Hurricanes.pickle")
Hurricanes

,Name,Dates as aCategory 5,Duration as aCategory 5,Sustainedwind speeds,Pressure,Areas affected,Deaths,Damage(USD),Refs
0,"""Cuba""","October 19, 1924",12 hours,165 mph (270 km/h),910 hPa (26.87 inHg),"Central America, Mexico, CubaFlorida, The Bahamas",90,NaN,[12]
1,"""San Felipe IIOkeechobee""","September 13–14, 1928",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Lesser Antilles, The BahamasUnited States East Coast, Atlantic Canada",4000,$100 million,NaN
2,"""Bahamas""","September 5–6, 1932",1 day,160 mph (260 km/h),921 hPa (27.20 inHg),"The Bahamas, Northeastern United States",16,NaN,NaN
3,"""Camagüey""","November 5–8, 1932",3 days 6 hours,175 mph (280 km/h),915 hPa (27.02 inHg),"Lesser Antilles, Jamaica, Cayman IslandsCuba, The Bahamas, Bermuda",3103,$40 million,NaN
4,"""Cuba–Brownsville""","August 30, 1933",12 hours,160 mph (260 km/h),930 hPa (27.46 inHg),"The Bahamas, Cuba, FloridaTexas, Tamaulipas",179,$27.9 million,NaN
5,"""Tampico""","September 21, 1933",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Jamaica, Yucatán Peninsula",184,$5 million,NaN
6,"""Labor Day""","September 3, 1935",18 hours,185 mph (295 km/h),892 hPa (26.34 inHg),"The Bahamas, Florida, GeorgiaThe Carolinas, Virginia",408,NaN,NaN
7,"""New England""","September 19–20, 1938",18 hours,160 mph (260 km/h),940 hPa (27.76 inHg),"Eastern United States, Southwestern Quebec",682,$306 million,NaN
8,Carol,"September 3, 1953",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Bermuda, New England, Atlantic Canada",5,$2 million,NaN
9,Janet,"September 27–28, 1955",18 hours,175 mph (280 km/h),914 hPa (26.99 inHg),"Lesser Antilles, Central America",1023,$65.8 million,[13]


In [47]:
Hurricanes = pd.read_pickle("Pickles/Hurricanes.pickle")
Hurricanes.drop(Hurricanes.tail(1).index,axis=0, inplace=True)
Hurricanes.drop('Refs', axis=1, inplace=True)
Hurricanes.columns = ["Name","During","Duration","Windspeed","Pressure","At","Deaths","Damage"]

Hurricanes.At = Hurricanes.At.apply(lambda x: re.findall('[A-Z][^A-Z]*', x)) 
# write markdown about method (maybe even make tree)

LastPlace = []
for places in Hurricanes.At:
    placeholder = []
    LastPlace.append([])
    for place in places:
        if len(placeholder) == 0:
            placeholder.append(place)
            if (' ' not in place) or (',' in place):
                LastPlace[-1].append(place.strip(', '))
                placeholder = []
            continue
                
        if ' ' == placeholder[-1][-1]:
            placeholder.append(place)
        
        if ',' in placeholder[-1] or ' ' not in placeholder[-1]:
            LastPlace[-1].append(' '.join(placeholder).strip(', '))
            placeholder = []
            
Hurricanes.At = pd.Series(LastPlace)
# The same thing happens for the Hurricane San Felipe II Okeechobee, but... it easier to just correct one data point
Hurricanes.iloc[1,0] = 'San Felipe II Okeechobee'

In [48]:
Hurricanes.head(5)

,Name,During,Duration,Windspeed,Pressure,At,Deaths,Damage
0,"""Cuba""","October 19, 1924",12 hours,165 mph (270 km/h),910 hPa (26.87 inHg),"[Central America, Mexico, Cuba, Florida, The Bahamas]",90,NaN
1,San Felipe II Okeechobee,"September 13–14, 1928",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"[Lesser Antilles, The Bahamas, United States East Coast, Atlantic Canada]",4000,$100 million
2,"""Bahamas""","September 5–6, 1932",1 day,160 mph (260 km/h),921 hPa (27.20 inHg),"[The Bahamas, Northeastern United States]",16,NaN
3,"""Camagüey""","November 5–8, 1932",3 days 6 hours,175 mph (280 km/h),915 hPa (27.02 inHg),"[Lesser Antilles, Jamaica, Cayman Islands, Cuba, The Bahamas, Bermuda]",3103,$40 million
4,"""Cuba–Brownsville""","August 30, 1933",12 hours,160 mph (260 km/h),930 hPa (27.46 inHg),"[The Bahamas, Cuba, Florida, Texas, Tamaulipas]",179,$27.9 million


In [49]:
## Damage -> Dollars (int) 
## Pressure -> Pa (int)
## Windspeed -> kmph (int)
## Duration -> Minutes (int)
## During -> dd/mm/yyyy (Date)

In [50]:
Hurricanes.Deaths = Hurricanes.Deaths.apply(int)

In [51]:
Hurricanes.Damage = Hurricanes.Damage.apply(lambda x: unicodedata.normalize("NFKD", str(x)))

dmg_conversions = dict(million=float(10**6),billion=float(10**9))
dmg_new = []

for damage in Hurricanes.Damage:
    if damage == 'nan':
        dmg_new.append(np.nan)
        continue
        
    cost = damage.split(' ')[0]
    unit = damage.split(' ')[-1]
    
    if unit not in dmg_conversions:
        dmg_new.append(damage)
        print(f'"{damage}" is one of the damage values. It does not match any known units.')
        continue
    
    if '$' not in cost:
        dmg_new.append(damage)
        print(f'This cost: {cost} is not in USD. The cost in dollars will not be calculated.')
        continue
    
    cost = float(cost.strip()[1:])
    dmg_new.append(cost*dmg_conversions[unit])
    
Hurricanes.Damage = pd.Series(dmg_new)
Hurricanes

"Extensive" is one of the damage values. It does not match any known units.


,Name,During,Duration,Windspeed,Pressure,At,Deaths,Damage
0,"""Cuba""","October 19, 1924",12 hours,165 mph (270 km/h),910 hPa (26.87 inHg),"[Central America, Mexico, Cuba, Florida, The Bahamas]",90,NaN
1,San Felipe II Okeechobee,"September 13–14, 1928",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"[Lesser Antilles, The Bahamas, United States East Coast, Atlantic Canada]",4000,100000000.0
2,"""Bahamas""","September 5–6, 1932",1 day,160 mph (260 km/h),921 hPa (27.20 inHg),"[The Bahamas, Northeastern United States]",16,NaN
3,"""Camagüey""","November 5–8, 1932",3 days 6 hours,175 mph (280 km/h),915 hPa (27.02 inHg),"[Lesser Antilles, Jamaica, Cayman Islands, Cuba, The Bahamas, Bermuda]",3103,40000000.0
4,"""Cuba–Brownsville""","August 30, 1933",12 hours,160 mph (260 km/h),930 hPa (27.46 inHg),"[The Bahamas, Cuba, Florida, Texas, Tamaulipas]",179,27900000.0
5,"""Tampico""","September 21, 1933",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"[Jamaica, Yucatán Peninsula]",184,5000000.0
6,"""Labor Day""","September 3, 1935",18 hours,185 mph (295 km/h),892 hPa (26.34 inHg),"[The Bahamas, Florida, Georgia, The Carolinas, Virginia]",408,NaN
7,"""New England""","September 19–20, 1938",18 hours,160 mph (260 km/h),940 hPa (27.76 inHg),"[Eastern United States, Southwestern Quebec]",682,306000000.0
8,Carol,"September 3, 1953",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"[Bermuda, New England, Atlantic Canada]",5,2000000.0
9,Janet,"September 27–28, 1955",18 hours,175 mph (280 km/h),914 hPa (26.99 inHg),"[Lesser Antilles, Central America]",1023,65800000.0


In [43]:
Hurricanes.Pressure = Hurricanes.Pressure.apply(lambda x: unicodedata.normalize("NFKD", str(x)))

str